In [1]:
import os

In [2]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = (CONFIG_FILE_PATH),
                 params_filepath = (PARAMS_FILE_PATH),
                 schema_filepath = (SCHEME_FILE_PATH)
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            all_params=params
        )

        return model_evaluation_config




In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import joblib
from mlProject.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config  = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)

        return (rmse, mae, r2)
    
    def evaluate_model(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        predicted = model.predict(test_x)

        rmse, mae, r2 = self.eval_metrics(test_y, predicted)

        scores = {'rmse': rmse, 'mae': mae, 'r2': r2}

        save_json(path=Path(self.config.metric_file_name), data=scores)

        
        


In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_trainer_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[2024-01-22 00:23:26,419: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-22 00:23:26,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-22 00:23:26,427: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-22 00:23:26,427: INFO: common: Created directory at artifacts]
[2024-01-22 00:23:26,435: INFO: common: Created directory at artifacts/model_evaluation]
[2024-01-22 00:23:26,675: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]
